# D) 2D linear magnetostatics

- **<font color='green'>[RUN & OBSERVE]</font>** $\rightarrow$ the cell should be run directly without modification
- **<font color='orange'>[RUN & PLAY]</font>** $\rightarrow$ the cell can be run directly, but some parameters should be changed interactively
- **<font color='red'>[FILL & RUN]</font>**    $\rightarrow$ the cell should be filled before being run
- **<font color='magenta'>[FILL & PLAY]</font>** $\rightarrow$ the cell should be filled, and then some parameters should be changed interactively.

_______
## 1) Equation

We want to solve the following weak formulation, i.e., find $a\in H^1_0 (\Omega)$ such that

$$ \forall v\in H^1_0(\Omega) , \quad \int_\Omega \text{curl}~ v  \cdot \nu \text{curl}~ a  = \int_\Omega v j $$

with $\nu$ the magnetic reluctivity, which is :
- $ \nu = 1/\mu_0 $ in air and conductors
- $ \nu = 1/(1000 \mu_0) $ in iron

and $j$ the current density that is :
- $ j = 0 $ in air and iron
- $ j= +J $ in positive conductor
- $ j = -J$ in negative conductor
  
with $J = 10\;A/mm^2$. 

________
## 2) Geometry

We consider a closed iron core with a winding. Without airgap, there is a risk of saturation !

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [1]:
from utils.myGeometries import gapedInductor
from ngsolve.webgui import Draw
mesh = gapedInductor(airgap = 0, h = 0.01) # no airgap
Draw(mesh.MaterialCF({"condN":1, "iron":2, "condP":3}), mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

We can get the labels of the regions...
|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [2]:
mesh.GetMaterials()   # region names

('air', 'iron', 'condP', 'condN')

... and boundaries.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [3]:
mesh.GetBoundaries()   # boundaries names

('out',
 'out',
 'out',
 'out',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '')

________
## 3) Linear formulation

### a) Physical properties

Assume first $\nu_{iron} = \frac{1}{1000 \mu0}$, and $J = 10 A/mm^2$.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [4]:
from numpy import pi
mu0 = 4e-7 * pi  # H/m, void magnetic permeability
J = 10e6         # A/m^2

### b) Function space

|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [5]:
from ngsolve import H1
fes = H1(mesh, order = 1, dirichlet = "out")

### c) Integral equation

**<font color='red'>Instructions à rappeler</font>**

|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [6]:
from ngsolve import grad, dx, LinearForm, BilinearForm, CoefficientFunction
R = CoefficientFunction( ((0,1),(-1,0)), dims = (2,2))
curl = lambda u : R * grad(u)

a, v = fes.TnT()
A = BilinearForm(fes)
A += 1/mu0 * curl(a) * curl(v) * dx("air|condP|condN")
A += 1/(1000*mu0) * curl(a) * curl(v) * dx("iron")
l = LinearForm(fes)
l += v * J * dx("condP") 
l += - v * J * dx("condN")

### d) Assemble and solve the system

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [7]:
K = A.Assemble().mat
f = l.Assemble().vec

from ngsolve import GridFunction
sol = GridFunction(fes)
sol.vec.data = K.Inverse(freedofs = fes.FreeDofs()) * f
Draw(sol)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

### e) Post-processing : flux density
We can compute the flux density $B = \text{curl}~a$.
|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [8]:
B = curl(sol)
Draw(B, mesh, 
     vectors={"grid_size" : 50, "offset" : 0.5 },
     settings = { "Objects" : { "Wireframe" : False, "Surface" : False }})

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fals…

BaseWebGuiScene

The flux density is far too high! The iron saturates at lower induction levels. We should therefore consider the nonlinear material model.